# Introduction to Pandas, Part 2

## Grouping 

Grouping (aka selecting) is a very common task.   Say, you had 10 participatns and each completed 3 conditions.  By "grouping" we mean taking considering each "group" of 10 participants seperately for each condition and calculating something, like an average.   That's just one example.  There are many others that can be done.    It's one of the most useful tools to be able to do easily on a dataset.  


## Classes

We've gone over variables and functions but haven't mentioned classes.   We won't be going over them in this module.  Making classes is a slightly a more advanced programming skill.   However, we've been using them already with Pandas.  

Classes provide a means of bundling data and functionality together.  They define both the data and useful things to do to the data.  Because they are together you can unlock some simple syntax that can do complicated things.


Say you wanted to sort and grab the first 10 rows.   

If you just used seperate functions 
```
sorted_df = sort_values(df,'column A')
top10 = head(df,10)

```

There's nothing wrong with that.   A lot of code is written that way.   But with classes the 

```
df.sort_values('column A').head(10)
```

There's a lot more stuff classes unlock. 

Let's start by loading the California baby names again.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# import seaborn as sns
# sns.set()

from IPython.display import display
import urllib.request
import os.path
import zipfile

data_url = "https://www.ssa.gov/oact/babynames/state/namesbystate.zip"
local_filename = "../week_2/babynamesbystate.zip"
if not os.path.exists(local_filename): # if the data exists don't download again
    with urllib.request.urlopen(data_url) as resp, open(local_filename, 'wb') as f:
        f.write(resp.read())

zf = zipfile.ZipFile(local_filename, 'r')

ca_name = 'CA.TXT'
field_names = ['State', 'Sex', 'Year', 'Name', 'Count']
with zf.open(ca_name) as fh:
    babynames = pd.read_csv(fh, header=None, names=field_names)

babynames.sample(5)

# Grouping and Aggregating



To understand how groupby works, consider the visual diagram below. The `groupby` function clusters rows from the original dataframe into groups (which I call subframes). The `agg` function then condenses each subframe into a single representative row using the provided function f.

![image](groupby_max.png)

Let's actually build this DataFrame and play with it interactively to understand `groupby` a little better:

In [ ]:
d = pd.DataFrame(dict(x=[3,1,4,1,5,9,2,5,6], y=[12,7,3,2,7,3,8,6,7]), index=list('ABCABCACB') )
d

Groupby returns a funny thing.   We've been using pandas where the functinos we call return dataframes.   So they get rendered in our environment nicely and make sense.   The groupby is an intermediate form or "object" that doesn't have a nice render defined. 

In [ ]:
dgb = d.groupby(d.index)
dgb

That output just says that there is a groupby object defined.   Not useful at all for you.  It's much more useful to use some functions that return parts of the groupby. For example, how were the groups labeled.   That's stored in the groupby object as a "Dictionary".    

In [ ]:
dgb.groups

This isn't a funcion.   It's a piece of data stored about the groupby object.  That's why we don't use groups()  

In [ ]:
dgb.groups()

But we can access the data in this dictionary just as if it was any other dictionary

In [ ]:
dgb.groups['A']

A very useful function is to get the values in a specific group

In [ ]:
dgb.get_group('A')

Another useful thing is that groupby() outputs can be looped over in a fairly straightforward way.  This is useful for inspecting data and for debugging

In [ ]:
#Loop over the groups, printing the name and the values
for name, group in dgb:
    print(name)
    print(group)

In [ ]:
#Loop over the groups, printing the name and the values, but with a slightly prettier output

for name, group in dgb:
    print(f"Group: {name}")
    print(group)
    print() # print a blank line
    

---

To test your understanding, try to interpret the result of the code below.

In [ ]:
babynames[ ["Year","Count"] ].groupby("Year").agg(max).plot();

For reference, the first 5 values from the plot above are:

In [ ]:
babynames[ ["Year","Count"] ].groupby("Year").agg(sum).head(5)

## groupby warning

It is easy to get unexpected things.  Groupby is a powerful command that can do a lot of data manipulations but it is easy to get results you don't intend. 

Let's switch dataset quickly and look at the US Presidential elections again.  Suppose you want to know the best election result that each political party achieved.  

In [ ]:
elections = pd.read_csv("elections.csv")
elections.sample(5)

We have to be careful when using aggregation functions. For example, the code below might be misinterpreted to say that Woodrow Wilson ran for election in 2016, or that William Taft won the Presidential election in 2020 for the Republicans.   Why is this happening?

In [ ]:
elections.groupby("Party").agg(max)

In [ ]:
elections.groupby("Party").agg(max).head(5)

In [ ]:
elections.groupby("Party").get_group("American")

### groupby fix

Next we'll write code that properly returns _the best result by each party_. That is, each row should show the Year, Candidate, Popular Vote, Result, and % for the election in which that party saw its best results (rather than mixing them and treating every column independently as in the example above), here's what the first rows of the correct output should look like:


In [ ]:
elections_sorted_by_percent = elections.sort_values("%", ascending=False)
elections_sorted_by_percent.head(5)

We used the agg(max) method to  combine above. Instead, now that we have a sorted table it is now simply a matter of taking the first() row and ignoring all the others.    

In [ ]:
elections_sorted_by_percent = elections.sort_values("%", ascending=False)
elections_sorted_by_percent.groupby("Party").get_group("Democratic").head(5)

In [ ]:
    
elections_sorted_by_percent = elections.sort_values("%", ascending=False)
elections_sorted_by_percent.groupby("Party").first()

You'll soon discover that with Pandas' rich tool set, there's typically more than one way to get to the same answer. Each approach has different tradeoffs in terms of readability, performance, memory consumption, complexity and more.  It will take some experience for you to develop a sense of which approach is better for each problem, but you should in general try to think if you can at least envision a different solution to a given problem, especially if you find your current solution to be particularly convoluted or hard to read.


---

## Other groupby Features

### groupby.size()

In [ ]:
elections.groupby("Party")

In [ ]:
#size returns a Series giving the size of each group
elections.groupby("Party").size().head(15)

### groupby.filter()

Useful to know it exists. Can filter by criteria within a group.    But I won't talk about it.  

In [ ]:
# filter gives a copy of the original DataFrame where row r is included
# if its group obeys the given condition
#
# Note: Filtering is done per GROUP, not per ROW.
elections.groupby("Year").filter(lambda sf: sf["%"].max() < 45)

### groupby.sum(), groupby.mean(), etc.

As an alternative to groupby.agg(sum), we can also simply do groupby.sum().

In [ ]:
elections.groupby("Year").agg(sum).head()

In [ ]:
elections.groupby("Year").sum().head()

The same applies for many other common operations.

In [ ]:
elections.groupby("Year").agg(max).head()

In [ ]:
elections.groupby("Year").max().head()

In [ ]:
#elections.groupby("Year").mean().head()
elections.groupby("Year").median().head()
#elections.groupby("Year").max().head()

# Grouping by more than one thing


Groupby() is not just limited to 1 column name you can create an index with multiple columns too. 

This uses jargon alert: **Hierarchical indexing**




In [ ]:
elections.groupby(["Year", "Result"]).agg(max).head(10)

In [ ]:
babynames.groupby(["Sex", "Year"]).agg(max).head(6)

Suppose we want to build a table showing the most babies born with the same name for each sex in each year. One way is to groupby using both columns of interest.

In [ ]:
#Not quite the expected output. 
babynames.groupby(["Year", "Sex"]).agg(max).tail(10)

Anything funny about that output?

In [ ]:
babynames.groupby(["Year", "Sex"]).agg(sum).head(6)

#Slicing

In order to select things in the multi index you need to use python "slice" syntax.  We did that last week.  But it gets fiddly with a multi index. 

Remember how we could use start:end to select a range?  You can do the same with a multi index.   But it's funky because a multiple index has multiple things.   


In [7]:
babyGrouped = babynames.groupby(["Year", "Sex"],).agg(max)





In [ ]:
#Slice Example
babyGrouped.loc[ (2010,'M') ,slice(None)]

babyGrouped.loc[ (2010,'M'):(2012,'F') ,slice(None)]


This is rather a tricky syntax to master.  But it will be very useful because you often want to group using the values in multiple columns.

## Resetting the index

Sometimes instead of using the a heirarchical multi index it's easier to just reset the index.  That puts the index values back into the columns. 

### WARNING Output vs. changing things

Most of these commands are not -changing- the dataframe.  You need to specifically do that. 

In [ ]:
babyGrouped.reset_index()


In [ ]:
babyGrouped

In [11]:
babyGroupedReset = babyGrouped.reset_index()
babyGroupedReset

In [ ]:
babyGrouped.reset_index(inplace=True)

## EZ plotting

Pandas has a some built in plotting functions that can be used to get a quick view on the data.   They only create simple plots and they don't have powerful features like the Seaborn package we will learn later.  They can be fairly simplistic and not visual appealing and difficult for someone who is not familiar with the dataset to interpret.  They aren't really publication/report quality.  But they are fairly useful when you are doing data expoloration and just want a quick visual representation.   

In [ ]:
#Let's try just plotting our babynames dataset:
babynames.plot()

Woah.  What is that?  Well it's just plotting the series by the index.   The index is just the raw line of the dataset.   It gives a quick view.  But it's totally un-interpretable. 

What might be more useful?   Well we have an variable "year" in our dataset.  Let's plot the data 

In [ ]:
#Plotting by year
#First group by "Year" then take the sum of al the counts ()
babynames.groupby(["Year"]).agg(sum).plot()

Let's look at what went into that dataset to plot:

In [ ]:
#Let's take a look at the first 10 rows of our dataset
babynames.groupby(["Year"]).agg(sum).head(10)

Ok So that's crazy right?   What is going on?  sum() is like "+"  and we've used "+" alot for strings in previous classes.   What + and a string does is to concatenate (stick together) the strings. Creating basically a nightmare unusable value in the column. Luckily when you go to "plot" it only plots numeric values, it ignores the strings. 
Let's be a bit more direct and choose a specific outcome column


In [ ]:
#Let's make the same plot but only select the "Count column"
#Note you I enclosed "Count" in []  so it's [ ["Count"] ]  that returns a dataframe instead of a series.  It works for a series as well
#But you lose the column label so that can get confusing. 
babynames.groupby(["Year"])[["Count"]].agg(sum)

In [ ]:
#Another way to do the same thing is to use a dictionary to specify the column and the aggregation function
#This is useful if you want to aggregate multiple columns, or if you want to use different aggregation functions for different columns
babynames.groupby(["Year"]).agg({"Count":sum})

In [ ]:
#As above, but here is showing the syntax for returning multiple different aggregations for the same column
#In this case, we are returning the sum, min, and max of the "Count" column
#This is extremely useful in practice
babynames.groupby(["Year"]).agg({"Count":[sum,min,max]})

In [ ]:
#Now let's make the plot. 

babynames.groupby(["Year"])[["Count"]].agg(sum).plot()


## Pivot Tables

Pivot tables are another way to reformat and group data.  They are a different syntax that some find more natural. Especially if you've used the function in Excel before.  When grouping more than one thing they can be easier to work with.    

Suppose we want to build a table showing the total number of babies born of each sex in each year. One way is to groupby using both columns of interest.

In [ ]:
babynames.groupby(["Year", "Sex"]).agg(sum).head(6)

Another way is to use a "Pivot Table".   

In [ ]:
babynames_pivot = babynames.pivot_table( index='Year', columns='Sex', values=['Count'], aggfunc=np.max, )
babynames_pivot.head()

THat line of code is getting complicated and hard to read.   Let's use formatting to make it simpler.  Below is exactly the same code

In [ ]:
babynames_pivot = babynames.pivot_table(
    index='Year', # the rows (turned into index)
    columns='Sex', # the column values
    values=['Count'], # the field(s) to processed in each group
    aggfunc=np.sum, # group operation
)
babynames_pivot.tail()

Further, with data structured this way making comparisons plots becomes straightforward.



In [ ]:
babynames_pivot = babynames.pivot_table(
    index='Year', # the rows (turned into index)
    columns='Sex', # the column values
    values=['Count'], # the field(s) to processed in each group
    aggfunc=np.sum, # group operation
)
babynames_pivot.plot()



Notice anything in that plot?

### Excercises:  

## Grouping

More careful look at the most popular 2018 name in California.
In last weeks tasks for popular names, we didn't take into account the unlikely possibility that the most popular name was actually spread across both birth sexes. For example, what if in the table below it turns out that there were 300 female Noahs born in CA in 2018? In that case, Noah would actually be the most popular.

Since our queries are getting pretty long, I've stuck them inside parentheses which allows us to spread them over many lines.

```python
#This line of code can has lots of function in a row and can make it hard to read 
firstthing.nextFunction().anotherFunction()

#Here is a the exact same code, just formated in a group for easier reading 
( #<--- start a a group of code. 
firstthing   #<---- start the first experssion
    .nextFunction() #<--- do something
    .anotherFunction() #<--- do something else
) #<--- end the group
```

In [ ]:

#
#
# ( <--- start a a group of code. 
# firstthing   <---- start the first experssion
# .nextFunction() <--- do something
# .anotherFunction() <--- do something else
# ) <--- end the group



(
babynames[babynames["Year"] == 2018]  # Select the rows where "year" is 2018
    .sort_values(by = "Count", ascending = False)  # sort the names by the count
    .head(5)  # take the top 5
)

Try to add a code into the operation above so that each row represents the sum of both male and female babies born in 2018 with the same name. To do this, fill in the ... below.



In [ ]:
(
babynames[babynames["Year"] == 2018]
    ...
    ...   
    .sort_values(by = "Count", ascending = False)
    .head(5)
)

Here is another example of AI generated code. This code runs and produces an output.    Try and figure out what is wrong with this output. 

You don't have to fix it. 

In [ ]:

#AI Response 1: Microsoft CO-Pilot
#Here is an AI generate coding for the question: "Can you plot the percentage of babies named "John" over time?"
#This is wrong.  Run the code and determine why? 

#Can you plot the percentage of babies named "John" over time?
babynames[babynames["Name"] == "John"].groupby("Year").agg(sum).plot()



Can you make a table that lists the #1 names for the years 2000-2010?

How many  babys in total were born in 1940? in 2020?

Can you determine the most used name in total over all time?  For males and females seperately?  What does this tell you?
 


Before we were looking at the differences in popularity.  Considering the total numbers of babys born does this have any implications for how we should inpterpret our previous analysis above? Starting from the code from above to plot the total number of babies.
```python
#Try ploting this use .plot() instead of selecting a year
(
    babynames
    .groupby(["Year"])[["Count"]]
    .agg(sum)
    .plot()
)

```

Can you make a plot of the number of unique names given over the years? This one is harder and requires using a function we didn't use above, the nunique() function counts the number of unique entries in a list. 

Given the most poopular names you found above, Can you plot the number of babies using those names over time?  

What other things can you think of?   Try and think of a question. 



